In [107]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch
from torch import nn
import cvxpy as cp
import pandas as pd
from typing import Union
import abc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [108]:
# ---------------------------
# Helper Functions & Globals
# ---------------------------
Activation = Union[str, nn.Module]
_str_to_activation = {
    'relu': nn.ReLU(),
    'tanh': nn.Tanh(),
    'leaky_relu': nn.LeakyReLU(),
    'sigmoid': nn.Sigmoid(),
    'selu': nn.SELU(),
    'softplus': nn.Softplus(),
    'identity': nn.Identity(),
}

def build_mlp(
        input_size: int,
        output_size: int,
        n_layers: int,
        size: int,
        activation: Activation = 'tanh',
        output_activation: Activation = 'identity',
):
    if isinstance(activation, str):
        activation = _str_to_activation[activation]
    if isinstance(output_activation, str):
        output_activation = _str_to_activation[output_activation]
    layers = []
    in_size = input_size
    for _ in range(n_layers):
        layers.append(nn.Linear(in_size, size))
        layers.append(activation)
        in_size = size
    layers.append(nn.Linear(in_size, output_size))
    layers.append(output_activation)
    return nn.Sequential(*layers)

device = None
def init_gpu(use_gpu=True, gpu_id=0):
    global device
    if torch.cuda.is_available() and use_gpu:
        device = torch.device("cuda:" + str(gpu_id))
        print("Using GPU id {}".format(gpu_id))
    else:
        device = torch.device("cpu")
        print("GPU not detected. Defaulting to CPU.")

def from_numpy(*args, **kwargs):
    return torch.from_numpy(*args, **kwargs).float().to(device)

def to_numpy(tensor):
    return tensor.to('cpu').detach().numpy()

# ---------------------------
# Fairness Measure & Optimization Solver
# ---------------------------
def AlphaFairness(util, alpha):
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha)/(1-alpha))

def solve_optimization(gainF, risk, cost, alpha, Q):
    """
    Solves the following optimization problem over the full decision vector d ∈ ℝⁿ:
    
    \[
    \begin{aligned}
    \max_{d \ge 0} \quad & W(d) = W\bigl(gainF \cdot risk \cdot d\bigr)\\[1mm]
    \text{s.t.} \quad & \sum_{i=1}^{n} cost_i \, d_i \le Q.
    \end{aligned}
    \]
    
    Returns the optimal decision vector \( d \) and the fairness objective value
    computed as
    \[
    F(d) = \text{AlphaFairness}\bigl(gainF \cdot risk \cdot d,\, \alpha\bigr).
    \]
    """
    # Convert inputs to numpy arrays if needed.
    gainF = gainF.detach().cpu().numpy() if isinstance(gainF, torch.Tensor) else gainF
    risk = risk.detach().cpu().numpy() if isinstance(risk, torch.Tensor) else risk
    cost = cost.detach().cpu().numpy() if isinstance(cost, torch.Tensor) else cost
    risk = np.clip(risk, 0.001, None)
    gainF, risk, cost = gainF.flatten(), risk.flatten(), cost.flatten()
    
    d = cp.Variable(risk.shape, nonneg=True)
    utils = cp.multiply(cp.multiply(gainF, risk), d)
    constraints = [d >= 0, cp.sum(cp.multiply(cost, d)) <= Q]
    
    if alpha == 1:
        objective = cp.Maximize(cp.sum(cp.log(utils)))
    elif alpha == 0:
        objective = cp.Maximize(cp.sum(utils))
    elif alpha == 'inf':
        t = cp.Variable()
        objective = cp.Maximize(t)
        constraints.append(utils >= t)
    else:
        objective = cp.Maximize(cp.sum(utils**(1-alpha))/(1-alpha))
    
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False, warm_start=True)
    if prob.status != cp.OPTIMAL:
        print("Warning: Problem status =", prob.status)
    optimal_decision = d.value
    optimal_value = AlphaFairness(optimal_decision * gainF * risk, alpha)
    return optimal_decision, optimal_value


In [109]:
# ---------------------------
# Data Loading & Preprocessing
# ---------------------------
# (Assume helper functions such as get_all_features are defined in your modules)
import sys
sys.path.insert(0, 'E:\\User\\Stevens\\Code\\The Paper\\algorithm')

from myutil import *
from features import get_all_features

alpha, Q = 2, 1000

df = pd.read_csv('data/data.csv')
df = df.sample(n=1000, random_state=42)

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 
    'gagne_sum_tm1', 'gagne_sum_t', 'risk_score_percentile', 'screening_eligible', 
    'avoidable_cost_mapped', 'propensity_score', 'g_binary', 'g_continuous', 
    'utility_binary', 'utility_continuous'
]
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# Define the inputs for decision-focused learning.
# Here the entire dataset represents one decision problem (n dimensions).
feats = df_feature[get_all_features(df_feature)].values
risk = df_stat['risk_score_t'].values.clip(0.001)      # true r ∈ ℝⁿ
gainF = df_stat['g_continuous'].values.clip(0.1)         # gain vector ∈ ℝⁿ
cost = np.ones(risk.shape)                               # cost vector ∈ ℝⁿ
race = df_stat['race'].values

scaler = StandardScaler()
feats = scaler.fit_transform(feats)

# For this problem, we treat the entire vector as one instance.
# (If desired, one might use cross-validation or different splits; here we split into train/test.)
train_feats, test_feats, train_risk, test_risk, train_gainF, test_gainF, train_cost, test_cost = train_test_split(
    feats, risk, gainF, cost, test_size=0.3, random_state=42)


In [110]:
# ---------------------------
# Model Definitions
# ---------------------------
class FairnessPredictor(nn.Module):
    """
    Predictor (C model) that maps features x to a predicted parameter \(\hat{r}\) (a vector of size n).
    """
    def __init__(self, input_dim, c_n_layers=0, c_layer_size=64, learning_rate=0.005, weight_decay=0.01,
                 activation="tanh", output_activation="relu"):
        super(FairnessPredictor, self).__init__()
        self.input_dim = input_dim
        # If c_n_layers==0, use a linear model.
        if c_n_layers == 0:
            self.model = nn.Linear(input_dim, 1)
        else:
            self.model = build_mlp(input_size=input_dim,
                                   output_size=1,
                                   n_layers=c_n_layers,
                                   size=c_layer_size,
                                   activation=activation,
                                   output_activation=output_activation)
        self.model.to(device)
        self.loss = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate, weight_decay=weight_decay)
        
    def forward(self, x):
        # x is of shape (n, input_dim), output is (n,)
        return self.model(x).squeeze(-1)
    
    def predict(self, x):
        self.eval()
        with torch.no_grad():
            return self.forward(x)

class LancerSurrogateFairness(nn.Module):
    """
    Surrogate (LANCER) model that predicts the fairness loss (i.e. regret)
    from the squared error \((r - \hat{r})^2\).
    """
    def __init__(self, lancer_n_layers=2, lancer_layer_size=64, learning_rate=0.001, weight_decay=0.01,
                 activation="relu", output_activation="relu"):
        super(LancerSurrogateFairness, self).__init__()
        self.model = build_mlp(input_size=1,
                               output_size=1,
                               n_layers=lancer_n_layers,
                               size=lancer_layer_size,
                               activation=activation,
                               output_activation=output_activation)
        self.model.to(device)
        self.loss = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate, weight_decay=weight_decay)
        
    def forward(self, pred_r, true_r):
        # Compute elementwise squared error, then output a scalar prediction per element.
        diff = (true_r - pred_r)**2
        if diff.dim() == 1:
            diff = diff.unsqueeze(1)
        # The model outputs a value per element; we average over all elements.
        return self.model(diff).squeeze(-1).mean()
    
    def forward_theta_step(self, pred_r, true_r):
        # Average surrogate output over all elements.
        return self.forward(pred_r, true_r)
    
    def update(self, pred_r, true_r, f_hat):
        # f_hat is a scalar target (the fairness loss/regret computed from the optimization problems).
        prediction = self.forward(pred_r, true_r)
        self.optimizer.zero_grad()
        loss = self.loss(prediction, f_hat)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [111]:
# ---------------------------
# LANCER Learner for Fairness (No Mini-batching)
# ---------------------------
class LancerLearnerFairness:
    def __init__(self, predictor: FairnessPredictor, surrogate: LancerSurrogateFairness, z_regul=0.0):
        self.predictor = predictor
        self.surrogate = surrogate
        self.z_regul = z_regul  # weight on the MSE loss for predictor update

    def initial_fit(self, feats, true_risk, c_epochs_init=30):
        """Warm-start the predictor using the full dataset (no mini-batching)."""
        self.predictor.train()
        X = from_numpy(feats)
        r_true = from_numpy(true_risk)
        for epoch in range(c_epochs_init):
            self.predictor.optimizer.zero_grad()
            r_pred = self.predictor(X)
            loss = self.predictor.loss(r_pred, r_true)
            loss.backward()
            self.predictor.optimizer.step()
            print(f"Initial fit epoch {epoch+1}, Predictor MSE Loss: {loss.item():.4f}")

    def train_surrogate(self, feats, true_risk, gainF, cost, alpha, Q, lancer_max_iter=5):
        """Train the surrogate (LANCER) model on the entire dataset."""
        self.surrogate.train()
        X = from_numpy(feats)
        r_true = from_numpy(true_risk)
        with torch.no_grad():
            r_pred = self.predictor(X)
        # Convert to numpy arrays.
        r_true_np = r_true.cpu().numpy().flatten()
        r_pred_np = r_pred.cpu().numpy().flatten()
        gainF_np = gainF.flatten()
        cost_np = cost.flatten()
        # Solve optimization with true risk:
        _, fairness_true = solve_optimization(gainF_np, r_true_np, cost_np, alpha, Q)
        # Solve optimization with predicted risk:
        pred_sol, _ = solve_optimization(gainF_np, r_pred_np, cost_np, alpha, Q)
        # Evaluate predicted fairness objective using true risk and the decision from predicted risk.
        pred_obj = AlphaFairness(gainF_np * r_true_np * pred_sol, alpha)
        f_hat = fairness_true - pred_obj  # target fairness loss (regret)
        f_hat_tensor = from_numpy(np.array([f_hat]))
        # Compute surrogate prediction over the full dataset.
        surrogate_output = self.surrogate.forward(r_pred, r_true)
        loss_val = self.surrogate.loss(surrogate_output, f_hat_tensor)
        self.surrogate.optimizer.zero_grad()
        loss_val.backward()
        self.surrogate.optimizer.step()
        return loss_val.item()

    def train_predictor(self, feats, true_risk, c_max_iter=5):
        """Update the predictor using surrogate feedback plus standard MSE loss on the full dataset."""
        self.predictor.train()
        X = from_numpy(feats)
        r_true = from_numpy(true_risk)
        total_loss = 0.0
        for _ in range(c_max_iter):
            r_pred = self.predictor(X)
            surrogate_loss = self.surrogate.forward_theta_step(r_pred, r_true)
            mse_loss = self.predictor.loss(r_pred, r_true)
            loss = surrogate_loss + self.z_regul * mse_loss
            self.predictor.optimizer.zero_grad()
            loss.backward()
            self.predictor.optimizer.step()
            total_loss += loss.item()
        return total_loss / c_max_iter

    def compute_regret(self, feats, true_risk, gainF, cost, alpha, Q):
        """
        Compute the normalized regret on the full dataset.
        
        For the full instance:
        - Solve the optimization problem with true risk to get opt_val.
        - Solve the optimization problem with predicted risk to get a decision, then compute
          the predicted fairness objective using true risk:
          
          \[
          \text{pred\_obj} = \text{AlphaFairness}(gainF \cdot r \cdot \mathbf{d}(\hat{r}), \alpha)
          \]
          
        - The normalized regret is:
          \[
          \text{Regret} = \frac{opt\_val - pred\_obj}{|opt\_val| + \epsilon}.
          \]
        """
        self.predictor.eval()
        X = from_numpy(feats)
        r_true = from_numpy(true_risk)
        r_pred = self.predictor(X)
        r_true_np = to_numpy(r_true).flatten()
        r_pred_np = to_numpy(r_pred).flatten()
        r_pred_np = np.clip(r_pred_np, 0.001, None)
        gainF_np = gainF.flatten()
        cost_np = cost.flatten()
        _, opt_val = solve_optimization(gainF_np, r_true_np, cost_np, alpha, Q)
        pred_sol, _ = solve_optimization(gainF_np, r_pred_np, cost_np, alpha, Q)
        pred_obj = AlphaFairness(gainF_np * r_true_np * pred_sol, alpha)
        regret = (opt_val - pred_obj) / (abs(opt_val) + 1e-7)
        return regret

    def training_loop(self, train_feats, train_risk, train_gainF, train_cost,
                      test_feats, test_risk, test_gainF, test_cost,
                      alpha, Q, n_iter=10, c_epochs_init=30, c_max_iter=5, lancer_max_iter=5, print_freq=1):
        # Warm-start the predictor.
        print("Warm-starting predictor...")
        self.initial_fit(train_feats, train_risk, c_epochs_init)
        for itr in range(n_iter):
            print(f"\nIteration {itr+1}")
            avg_surrogate_loss = 0.0
            for _ in range(lancer_max_iter):
                avg_surrogate_loss += self.train_surrogate(train_feats, train_risk, train_gainF, train_cost, alpha, Q, lancer_max_iter=1)
            avg_surrogate_loss /= lancer_max_iter
            avg_predictor_loss = self.train_predictor(train_feats, train_risk, c_max_iter)
            train_regret = self.compute_regret(train_feats, train_risk, train_gainF, train_cost, alpha, Q)
            test_regret = self.compute_regret(test_feats, test_risk, test_gainF, test_cost, alpha, Q)
            if (itr+1) % print_freq == 0:
                print(f"Iteration {itr+1}: Predictor Loss: {avg_predictor_loss:.4f}, Surrogate Loss: {avg_surrogate_loss:.4f}")
                print(f"Train Regret: {train_regret:.2f}%, Test Regret: {test_regret:.2f}%")


In [112]:
# Here we mimic extra runner parameters similar to the reference.
import random

def run_on_problem(params):
    # For reproducibility.
    seed = params["seed"]
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # In this fairness problem, we treat the entire instance as one problem.
    # (train/test split as above.)
    learner = LancerLearnerFairness(
        predictor=FairnessPredictor(input_dim=train_feats.shape[1],
                                    c_n_layers=params["c_n_layers"],
                                    c_layer_size=params["c_layer_size"],
                                    learning_rate=params["c_lr"],
                                    weight_decay=params["c_weight_decay"],
                                    activation="tanh",
                                    output_activation="relu"),
        surrogate=LancerSurrogateFairness(lancer_n_layers=params["lancer_n_layers"],
                                          lancer_layer_size=params["lancer_layer_size"],
                                          learning_rate=params["lancer_lr"],
                                          weight_decay=params["lancer_weight_decay"],
                                          activation="relu",
                                          output_activation="relu"),
        z_regul=params["z_regul"]
    )
    learner.training_loop(train_feats, train_risk, train_gainF, train_cost,
                          test_feats, test_risk, test_gainF, test_cost,
                          alpha, Q,
                          n_iter=params["n_iter"],
                          c_epochs_init=params["c_epochs_init"],
                          c_max_iter=params["c_max_iter"],
                          lancer_max_iter=params["lancer_max_iter"],
                          print_freq=params["print_freq"])

def main():
    # Define parameters directly as a dictionary
    params = {
        "seed": 42,
        "n_iter": 10,
        "print_freq": 1,
        "lancer_n_layers": 2,
        "lancer_layer_size": 100,
        "lancer_lr": 0.001,
        "lancer_weight_decay": 0.01,
        "lancer_max_iter": 5,
        "c_n_layers": 0,  # 0 for linear model
        "c_layer_size": 64,
        "c_lr": 0.005,
        "c_weight_decay": 0.01,
        "z_regul": 0.01,
        "c_max_iter": 5,
        "c_epochs_init": 30
    }
    print(params)
    run_on_problem(params)

if __name__ == '__main__':
    init_gpu(use_gpu=True, gpu_id=0)
    main()


Using GPU id 0
{'seed': 42, 'n_iter': 10, 'print_freq': 1, 'lancer_n_layers': 2, 'lancer_layer_size': 100, 'lancer_lr': 0.001, 'lancer_weight_decay': 0.01, 'lancer_max_iter': 5, 'c_n_layers': 0, 'c_layer_size': 64, 'c_lr': 0.005, 'c_weight_decay': 0.01, 'z_regul': 0.01, 'c_max_iter': 5, 'c_epochs_init': 30}
Warm-starting predictor...
Initial fit epoch 1, Predictor MSE Loss: 49.2716
Initial fit epoch 2, Predictor MSE Loss: 48.0012
Initial fit epoch 3, Predictor MSE Loss: 46.7925
Initial fit epoch 4, Predictor MSE Loss: 45.6450
Initial fit epoch 5, Predictor MSE Loss: 44.5583
Initial fit epoch 6, Predictor MSE Loss: 43.5319
Initial fit epoch 7, Predictor MSE Loss: 42.5647
Initial fit epoch 8, Predictor MSE Loss: 41.6551
Initial fit epoch 9, Predictor MSE Loss: 40.8006
Initial fit epoch 10, Predictor MSE Loss: 39.9984
Initial fit epoch 11, Predictor MSE Loss: 39.2455
Initial fit epoch 12, Predictor MSE Loss: 38.5391
Initial fit epoch 13, Predictor MSE Loss: 37.8765
Initial fit epoch 14, P